In [9]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load collected data
import pandas as pd

# Read the text file directly (it's already in CSV format)
df = pd.read_csv("/content/test.txt", header=None, names=['RSSI', 'Distance'])  # If no headers
# If your file has headers, use: df = pd.read_csv("your_text_file.txt")

X = df[['RSSI']].values  # Features (RSSI values)
y = df['Distance'].values  # Target (Distance values)

print("Features (X):", X)
print("Target (y):", y)

# Normalize RSSI
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save scaler values for use in ESP32 preprocessing
rssi_mean = scaler.mean_[0]
rssi_std = np.sqrt(scaler.var_[0])

# Train simple NN
model = keras.Sequential([
    keras.layers.Dense(8, activation='relu', input_shape=(1,)),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_scaled, y, epochs=50, validation_split=0.2)

# Save model and scaling values
model.save("rssi_distance_model.h5")
np.save("scaler_params.npy", [rssi_mean, rssi_std])



Features (X): [[-61]
 [-61]
 [-58]
 [-64]
 [-64]
 [-58]
 [-59]
 [-59]
 [-59]
 [-59]
 [-62]
 [-62]
 [-69]
 [-70]]
Target (y): [1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.5528 - val_loss: 1.8748
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 1.5373 - val_loss: 1.8648
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 1.5217 - val_loss: 1.8556
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 1.5054 - val_loss: 1.8463
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 1.4890 - val_loss: 1.8366
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 1.4723 - val_loss: 1.8267
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 1.4556 - val_loss: 1.8170
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 1.4383 - val_loss: 1.8075
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 1.4210 - val_loss: 1.7981
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 1.4036 - val_loss: 1.7889
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 1.3869 - val_loss: 1.7796
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 1.3701 - val_loss: 1

FileNotFoundError: [Errno 2] No such file or directory: 'rssi_dataset.csv'

In [13]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)



Saved artifact at '/tmp/tmpuhk81acd'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138695445561168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138695445559440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138695430390224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138695430390800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138695445559056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138695430390608: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [14]:
!xxd -i model.tflite > model_data.h
